In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, chdir
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent))
current_dir = getcwd()
print(current_dir)

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)

import numpy as np
from random import randint
from math import floor, ceil
import codecs

In [ ]:
from bulletin import root, default_input, default_output, agora, hoje, ontem, anteontem, dias_apos, dias_apos_label
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils import utils, static
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import trim_overspace
from bulletin.utils.normalize import normalize_text#normalize_hash, normalize_labels, , date_hash, normalize_number

In [ ]:
from datetime import datetime, date, timedelta
from tqdm import tqdm

exclusao_pathfile = join(root, 'database', 'casos_confirmados')

today = pd.to_datetime(date.today())
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(31)

In [ ]:
# FUNDAMENTAÇÃO (para a escolha do período)
# https://www2.ufjf.br/noticias/2021/04/20/infectologista-explica-riscos-de-reinfeccao-pelo-coronavirus-e-suas-variantes/

In [ ]:
def find_best_record(id_not_1, cns_1, cpf_1, diagnostico_1, evolucao_1, id_not_2, cns_2, cpf_2, diagnostico_2, evolucao_2):
    if ((cns_1 == None) and (cpf_1 == None)):
        if ((cns_2 != None) or (cpf_2 != None)):
            #TROCA AS FICHAS
            id_temp = id_not_1
            diag_temp = diagnostico_1
            evolucao_temp = evolucao_1


            id_not_1 = id_not_2
            diagnostico_1 = diagnostico_2
            evolucao_1 = evolucao_2


            id_not_2 = id_temp
            diagnostico_2 = diag_temp
            evolucao_2 = evolucao_temp
    
    return id_not_1, diagnostico_1, evolucao_1, id_not_2, diagnostico_2, evolucao_2

### 1. Determinação das FICHAS DUPLICADAS no CC

In [ ]:
cc = CasosConfirmados()

date = hoje

cc.load(f"cc_{date.strftime('%d_%m_%Y')}", compress=True)

cc.df.shape

In [ ]:
try: 
    cc.df = cc.df.drop(columns={'reinfeccao', 'reinfeccao_ids'})
except:
    pass
cc.df.shape

In [ ]:
cc_duplicados = cc.check_duplicates(keep=False)
cc_duplicados.reset_index(inplace=True)
cc_duplicados.groupby(['ids_len'])[['ids_len']].count()

In [ ]:
duplicados_only = cc_duplicados.drop_duplicates(subset='ids_str')
duplicados_only = duplicados_only.loc[duplicados_only['duplicated_cols'].str.contains('hash_mae')]
duplicados_only = duplicados_only.loc[duplicados_only['duplicated_ncols'] > 1]
# duplicados_only = duplicados_only.loc[duplicados_only['ids_len'] == 2]
duplicados_only = duplicados_only.sort_values(by='ids_len', ascending=True)
duplicados_only

### 2. Início do MÉTODO

In [ ]:
for i in tqdm(duplicados_only.index):
    list_verify = duplicados_only.loc[i, 'ids']
    df_temp = cc.df.loc[cc.df['id_notifica'].isin(list_verify)].sort_values('data_diagnostico')

    for j in range(0, len(list_verify) - 1, 1):
        #FICHA 1, original
        id_not_1 = df_temp.loc[df_temp.index[j], 'id_notifica']
        cns_1 = df_temp.loc[df_temp.index[j], 'cns']
        cpf_1 = df_temp.loc[df_temp.index[j], 'cpf']
        diagnostico_1 = df_temp.loc[df_temp.index[j], 'data_diagnostico']
        evolucao_1 = df_temp.loc[df_temp.index[j], 'evolucao']
        
        
        #FICHA 2, provável reinfeccao
        id_not_2 = df_temp.loc[df_temp.index[j+1], 'id_notifica']
        cns_2 = df_temp.loc[df_temp.index[j+1], 'cns']
        cpf_2 = df_temp.loc[df_temp.index[j+1], 'cpf']
        diagnostico_2 = df_temp.loc[df_temp.index[j+1], 'data_diagnostico']
        evolucao_2 = df_temp.loc[df_temp.index[j+1], 'evolucao']


        id_not_1, diagnostico_1, evolucao_1, id_not_2, diagnostico_2, evolucao_2 = find_best_record(id_not_1, cns_1, cpf_1, diagnostico_1, evolucao_1, id_not_2, cns_2, cpf_2, diagnostico_2, evolucao_2)

        
        diff = diagnostico_2 - diagnostico_1
        if (abs(diff.days) >= 90):
            cc.df.loc[cc.df['id_notifica'] == id_not_2, ('reinfeccao', 'reinfeccao_ids')] = (True, id_not_1)
        else:
            cc.df.loc[cc.df['id_notifica'] == id_not_2, ('reinfeccao', 'reinfeccao_ids')] = (False, id_not_1)
            duplicados_only.loc[duplicados_only['ids_str'].str.contains(str(id_not_2)), 'verify_duplicated'] = True

In [ ]:
temporario = cc.df.loc[(cc.df['reinfeccao'] == False) & (cc.df['reinfeccao_ids'].notna())]

qualificar = temporario.loc[temporario['evolucao'] == 2]
excluir = temporario.loc[temporario['evolucao'] != 2]
excluir.shape

In [ ]:
excluir['ano_caso'] = excluir['data_diagnostico'].dt.year
excluir.groupby('ano_caso')[['id_notifica']].count()

In [ ]:
qualificar.shape

In [ ]:
# 322 rows × 59 columns

In [ ]:
# raise

### 3. Ajuste das fichas no Notifica

In [ ]:
notifica = Notifica()
notifica.load('notifica', compress=False)
notifica.df = notifica.df.drop_duplicates('id', keep='last')
notifica.df.shape

In [ ]:
notifica.df.loc[notifica.df['id'].isin(excluir['id_notifica']), 'duplicated_cc'] = True
notifica.df['duplicated_cc'] = notifica.df['duplicated_cc'].fillna(False)
notifica.df.shape

### 4. Salvar os arquivos necessários

In [ ]:
path_db = join('C:\\', 'SESA', 'covid19datascience-dev', 'bulletin', 'database', 'casos_confirmados')

In [ ]:
duplicados_only.to_excel(join(path_db, 'relacao_duplicados_cc.xlsx'))

In [ ]:
qualificar.to_pickle(join(path_db, 'qualificar_obitos.pkl'))
qualificar.to_excel(join(path_db, 'qualificar_obitos.xlsx'))

In [ ]:
excluir.to_pickle(join(path_db, 'casos_excluir.pkl'))

In [ ]:
cc.save(f"cc_{date.strftime('%d_%m_%Y')}", replace=True, compress=True)

In [ ]:
notifica.save(replace=True, compress=False)